# Image analysis using scikit-image & napari

While tools such as FIJI, Imaris etc. can be very easy to use, using Python for image analysis can be very useful:
- Custom scripts allow you to analyse data in exactly the right way (not just what is implemented in a GUI)
- Python allows you to combine image analysis, machine learning and data science (e.g. one script for everything, from raw data to plots).
- Other neuroscience tools are written in Python, and so you can use functionality from these libraries (DeepLabCut, suite2p, BrainGlobe etc.)

## scikit image
[scikit](https://scikit-image.org) is an easy to use Python image processing library. It works well with numpy, it's well documented and easy to install across operating systems.

## napari
[napari](https://napari.org) is a multidimensional image viewer for Python. It allows for high-performance visualisation of data alongside analysis scripts. It's also easy to develop plugins for, so you can share your workflows with colleagues who may not know much Python.

## Load image as numpy array and inspect

## Load the data
We're using [pooch](https://www.fatiando.org/pooch/) to automate downloading the data, and check the hash.

In [ ]:
import tifffile
import pooch
from pathlib import Path

In [22]:
# Use pooch to fetch data if it hasn't already been downloaded
image_url = "https://gin.g-node.org/neuroinformatics/image-analysis-courses/raw/master/misc/DAPI.tif"
image_path = pooch.retrieve(image_url, path=Path.cwd().parent, fname="DAPI.tif", known_hash="6c4a065ef15f84adc5628b26f2b01e3694cea121425b8e349aab0c5b07582468", progressbar=True)

In [23]:
image = tifffile.imread(image_path)
print(image.dtype)
print(image.shape)

uint8
(1024, 1024)


## View image in napari

In [24]:
import napari

# create the viewer and display the image
viewer = napari.view_image(image)

## Use scikit-image to threshold

In [11]:
# Threshold using otsu's method and view
# Otsu (1979) IEEE Transactions on Systems, Man and Cybernetics. Vol SMC-9, No 1, p62
import skimage

thresholded = skimage.filters.threshold_otsu(image)
binary = image > thresholded
viewer.add_image(binary)

<Image layer 'binary' at 0x7f91981d66e0>

## Try preprocessing and other thresholding algorithms

In [12]:
# Smooth the image with a gaussian filter and try again
image_smoothed = skimage.filters.gaussian(image, sigma=5)
viewer.add_image(image_smoothed)

thresholded = skimage.filters.threshold_otsu(image_smoothed)
binary_smoothed = image_smoothed > thresholded
viewer.add_image(binary_smoothed)

<Image layer 'binary_smoothed' at 0x7f9174e1f6d0>

In [13]:
thresholded = skimage.filters.threshold_triangle(image_smoothed)
triangle_thresholded = image_smoothed > thresholded
viewer.add_image(triangle_thresholded)

<Image layer 'triangle_thresholded' at 0x7f9125b1abf0>

## Clean up the image to improve segmentation

In [14]:
# Start by removing small objects
min_object_size = 500  # Define a minimum object size to keep (in pixels)
cleaned_image = skimage.morphology.remove_small_objects(
    triangle_thresholded, min_size=min_object_size
)
viewer.add_image(cleaned_image)

<Image layer 'cleaned_image' at 0x7f9122776b00>

In [15]:
# Run a watershed
import scipy
import numpy as np

# Calculate distance transform
distance = scipy.ndimage.distance_transform_edt(cleaned_image)
viewer.add_image(distance)

# Find local max, and dilate to ensure one peak per cell
coords = skimage.feature.peak_local_max(
    distance, footprint=np.ones((50, 50)), labels=cleaned_image
)
mask = np.zeros(distance.shape, dtype=bool)
mask[tuple(coords.T)] = True
mask = skimage.morphology.binary_dilation(mask)
markers, _ = scipy.ndimage.label(mask)

# Run watershed
labels = skimage.segmentation.watershed(-distance, markers, mask=cleaned_image)
viewer.add_labels(labels)

<Labels layer 'labels' at 0x7f91221b4b50>

## Measure cell properties

In [19]:
props = skimage.measure.regionprops_table(
    labels,
    properties=(
        "area",
        "centroid",
        "area_bbox",
        "orientation",
        "axis_major_length",
        "axis_minor_length",
    ),
)

In [20]:
import pandas as pd

pd.DataFrame(props)

,area,centroid-0,centroid-1,area_bbox,orientation,axis_major_length,axis_minor_length
0,9347.0,47.351557,977.252594,11655.0,0.824051,123.574242,101.212164
1,24281.0,105.611713,373.657551,32835.0,-0.478749,210.500975,146.952951
2,21851.0,158.673882,526.202828,28938.0,1.130450,184.791408,151.148950
3,21562.0,182.137510,830.198497,27405.0,1.451295,207.053682,133.766799
4,17807.0,372.247038,317.961532,22140.0,-1.403451,183.185618,124.034410
5,20282.0,391.189232,615.939848,29754.0,-0.804173,204.286642,126.867983
6,26210.0,485.609958,111.854140,33755.0,-1.265196,221.906564,150.722677
7,20736.0,492.755208,839.006896,26082.0,-1.400560,192.486483,137.269558
8,23199.0,585.024527,341.651149,31603.0,-0.959365,202.158906,146.696404
9,18696.0,678.137035,56.627567,22625.0,-0.233333,189.539230,127.663598
